# Protein and ligand preparation for AutoDock Vina

### Import needed libraries

In [105]:
#conda install pdb2pqr, pdbfixer, biopython, meeko
import os
from Bio.PDB import PDBParser, PDBIO, Select
from rdkit import Chem
from rdkit.Chem import AllChem
from meeko import MoleculePreparation, PDBQTWriterLegacy
import subprocess

### Define some classes and helper functions

In [106]:
class ProteinSelect(Select):
    def __init__(self, ligand_resname):
        self.ligand_resname = ligand_resname

    def accept_residue(self, residue):
        """Select only protein residues, excluding ligand and water."""
        hetfield, _, _ = residue.get_id()
        if residue.get_resname().strip() == self.ligand_resname:
            return 0
        if hetfield.strip() != "" or residue.get_resname() == "HOH":
            return 0
        return 1

class LigandSelect(Select):
    def __init__(self, ligand_resname):
        self.ligand_resname = ligand_resname

    def accept_residue(self, residue):
        """Select only ligand residues."""
        return residue.get_resname().strip() == self.ligand_resname

def extract_protein_and_ligand(pdb_file, ligand_resname):
    """
    Extract protein and ligand separately from complex pdb.
    Args:
        pdb_file (str): Path to input PDB file containing protein-ligand complex.
        ligand_resname (str): Residue name of the ligand in the PDB file.
    Returns:
        protein_file (str): Path to output protein-only PDB file.
        ligand_file (str): Path to output ligand-only PDB file.
    """
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("complex", pdb_file)
    io = PDBIO()

    # Protein extraction
    io.set_structure(structure)
    protein_file = "protein_clean.pdb" 
    io.save(protein_file, ProteinSelect(ligand_resname)) #removed ligand_resname from ()

    # Ligand extraction
    io.set_structure(structure)
    ligand_file = "ligand_only.pdb"
    io.save(ligand_file, LigandSelect(ligand_resname)) #removed ligand_resname from ()

    print(f"[INFO] Extracted protein to {protein_file} and ligand to {ligand_file}")
    return protein_file, ligand_file

def prepare_ligand(ligand_pdb, output_pdbqt):
    """
    Prepare co-ligand for docking: add hydrogens, embed, optimize, export PDBQT.
    Args:
        ligand_pdb (str): Path to input ligand PDB file.
        output_pdbqt (str): Path to output prepared ligand PDBQT file.
    Returns:
        None
    """
    mol = Chem.MolFromPDBFile(ligand_pdb, removeHs=False)
    if mol is None:
        raise ValueError(f"Failed to read ligand from {ligand_pdb}")

    # Add hydrogens and generate 3D conformation, optimize geometry
    mol = Chem.AddHs(mol)
    #AllChem.EmbedMolecule(mol, AllChem.ETKDGv3())
    #AllChem.UFFOptimizeMolecule(mol)

    # Convert ligand to PDBQT via Meeko
    prep = MoleculePreparation()
    mol_setup = prep.prepare(mol)
    pdbqt_string, ok, err = PDBQTWriterLegacy.write_string(mol_setup[0])
    if not ok:
        raise ValueError(f"Failed generating PDBQT for ligand: {err}")

    with open(output_pdbqt, "w") as f:
        f.write(pdbqt_string)
    print(f"[INFO] Ligand prepared and saved to {output_pdbqt}")
    
def prepare_protein():
    """
    Prepare protein receptor for docking using Meeko (adds hydrogens, converts to PDBQT).
    """

    #Calculate the center of the ligand to set as box center
    lig = Chem.MolFromPDBFile("ligand_only.pdb", removeHs=False)
    conf = lig.GetConformer()
    xs, ys, zs = zip(*[list(conf.GetAtomPosition(i)) for i in range(lig.GetNumAtoms())])
    #store up to 3 decimal places
    xsc = round(sum(xs)/len(xs), 3)
    ysc = round(sum(ys)/len(ys), 3)
    zsc = round(sum(zs)/len(zs), 3)
    print(f"xsc = {xsc}, ysc = {ysc}, zsc = {zsc}")
    print(f"center_x = {sum(xs)/len(xs):.3f}")
    print(f"center_y = {sum(ys)/len(ys):.3f}")
    print(f"center_z = {sum(zs)/len(zs):.3f}")

    # The sed command to replace HIS with HID in the protein PDB file. This is done as Meeko doesn't recognise HIS
    sed_command = ["sed", "s/HIS/HID/g", "protein_clean.pdb"]
    # Run the command
    with open("input_HID.pdb", "w") as outfile:
        subprocess.run(sed_command, stdout=outfile, check=True)

    #Treat the file using AMBER's tleap, this converts the HIS to HID
    tleap_commands = """\
    source leaprc.protein.ff14SB
    mol = loadpdb input_HID.pdb
    savepdb mol output_HID.pdb
    quit
    """
    with open("tleap.in", "w") as f:
        f.write(tleap_commands)

    # Run tleap with the input file
    result = subprocess.run(["tleap", "-f", "tleap.in"], capture_output=True, text=True)
    print(result.stdout)

    # Construct the mk_prepare_receptor.py command with formatted box_center coordinates
    mk_prepare_cmd = [
        "mk_prepare_receptor.py",
        "--pdb", "output_HID.pdb",
        "-o", "final_1h1q_protein.pdbqt",
        "--box_size", "16", "16", "16",
        "--box_center", f"{xsc}", f"{ysc}", f"{zsc}"
    ]

    # Run the mk_prepare_receptor.py command
    mk_result = subprocess.run(mk_prepare_cmd, capture_output=True, text=True)
    print(mk_result.stdout)
    print(f"[INFO] Protein prepared and saved to final_1h1q_protein.pdbqt")

### Now define the main function

In [107]:
def main():
    pdb_file = "1h1q_chainA.pdb"  # Your input complex PDB
    ligand_resname = "2A6"  # Your ligand residue name in PDB

    protein_pdb, ligand_pdb = extract_protein_and_ligand(pdb_file, ligand_resname)

    ligand_pdbqt = "ligand_prepared.pdbqt"
    prepare_ligand(ligand_pdb, ligand_pdbqt)

    prepare_protein()

    print("[DONE] Receptor and ligand are ready for AutoDock Vina docking.")


### Call the main function

In [108]:
if __name__ == "__main__":
    main()

[INFO] Extracted protein to protein_clean.pdb and ligand to ligand_only.pdb
[INFO] Ligand prepared and saved to ligand_prepared.pdbqt
xsc = 6.145, ysc = 44.176, zsc = 50.828
center_x = 6.145
center_y = 44.176
center_z = 50.828


python(72196) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.
python(72197) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


-I: Adding /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/prep to search path.
-I: Adding /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/lib to search path.
-I: Adding /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/parm to search path.
-I: Adding /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/cmd to search path.
-f: Source tleap.in.

Welcome to LEaP!
(no leaprc in search path)
Sourcing: ./tleap.in
----- Source: /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/cmd/leaprc.protein.ff14SB
----- Source of /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/cmd/leaprc.protein.ff14SB done
Log file: ./leap.log
Loading parameters: /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/parm/parm10.dat
Reading title:
PARM99 + frcmod.ff99SB + frcmod.parmbsc0 + OL3 for RNA
Loading parameters: /Users/ganeshshahane/miniconda3/envs/TQ_test/dat/leap/parm/frcmod.ff14SB
Reading force field modification type file (frcmod)
Reading title:
ff14SB protein backbone and sidecha

python(72207) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.



Files written:
  final_1h1q_protein.pdbqt <-- static (i.e., rigid) receptor input file
boron-silicon-atom_par.dat <-- atomic parameters for B and Si (for autogrid)
    final_1h1q_protein.gpf <-- autogrid input file
final_1h1q_protein.gpf.pdb <-- PDB file to visualize the grid box

[INFO] Protein prepared and saved to final_1h1q_protein.pdbqt
[DONE] Receptor and ligand are ready for AutoDock Vina docking.


#### Some notes, and playing around

In [57]:
#!pdb2pqr --ff=AMBER --with-ph=7.4 --titration-state-method=propka protein_clean.pdb protein_clean_prepared.pqr

In [58]:
#!obabel protein_clean_prepared.pqr -O protein_clean_prepared.pdb

In [ ]:
#!pdbfixer protein_clean_prepared.pdb --ph=7.4 --add-atoms=all --add-residues --output protein_fixed.pdb

/Users/ganeshshahane/miniconda3/envs/TQ_test/lib/python3.12/pty.py:95: DeprecationWarning: This process (pid=73261) is multi-threaded, use of forkpty() may lead to deadlocks in the child.
  pid, fd = os.forkpty()


In [ ]:
#The following command was used to convert HIS to HID before tleap step
#sed 's/HIS/HID/g' protein_clean.pdb > input_HID.pdb

# Create the following tleap script and get the output_HID.pdb file
# tleap.in file content:
# source leaprc.protein.ff14SB
# mol = loadpdb input_HID.pdb
# savepdb mol output_HID.pdb
# quit
# Then run the following command to execute tleap
# !tleap -f tleap.in


#Then run the following command to generate final pdbqt file
#!mk_prepare_receptor.py --pdb output_HID.pdb -o final_1h1q_protein.pdbqt --box_size 20 20 20 --box_center 6.499 41.178 64.106